In [1]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data/data12632/


first_round_testin  first_round_traini	submit_example.csv


In [2]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

result.csv


In [3]:
! pip install catboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 62.4MB 478kB/s ta 0:00:012
    100% |████████████████████████████████| 7.1MB 3.1MB/s ta 0:00:011
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/aistudio/.cache/pip/wheels/4a/b7/7f/66e52d08955b288e3590645984c8e6fb1fc665754d722de6c6
Successfully built retrying


In [4]:
! pip install mlxtend

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 1.3MB 3.1MB/s ta 0:00:01
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))': /simple/matplotlib/
    100% |████████████████████████████████| 13.0MB 1.4MB/s ta 0:00:01
paddlepaddle-gpu 1.5.1.post97 has requirement matplotlib<=2.2.4; python_version < "3.6", but you'll have matplotlib 3.0.3 which is incompatible.
  Found existing installation: matplotlib 2.2.3
    Uninstalling matplotlib-2.2.3:
      Successfully uninstalled matplotlib-2.2.3


In [5]:
! pip install xgboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
    100% |████████████████████████████████| 142.8MB 224kB/s ta 0:00:013  48% |███████████████▍                | 68.7MB 1.8MB/s eta 0:00:41


In [6]:
! pip install lightgbm

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='mirrors.ustc.edu.cn', port=443): Read timed out. (read timeout=15)",)': /pypi/web/simple/lightgbm/
    100% |████████████████████████████████| 1.2MB 10.0MB/s ta 0:00:01


In [7]:
# ! pip install tqdm

In [8]:
# ! pip install auto-sklearn

In [9]:
# !pip list


In [10]:
from catboost import CatBoostClassifier
import catboost as cbt
import numpy as np
import pandas as pd
from mlxtend.classifier import StackingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import AdaBoostClassifier
# from tqdm import tqdm
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import time
import copy
import gc
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.tree import DecisionTreeClassifier

In [11]:
train = pd.read_csv('/home/aistudio/data/data12632/first_round_traini')
test = pd.read_csv('/home/aistudio/data/data12632/first_round_testin')
submit = pd.read_csv('/home/aistudio/data/data12632/submit_example.csv')

In [12]:
# 线下验证group 连续构造
for group in range(50):
    train['group_%s'%group] = (train.index+group)//50%120

# 线下验证group 随机构造
for group in range(50,100):
    name = 'group_%s'%group
    train[name] = 0
    kfold=KFold(n_splits=120, shuffle=True,random_state=group)
    split=kfold.split(train)
    i = 0
    for train_index,valid_index in split:
        train.iloc[valid_index,-1] = i
        i+=1

In [13]:
def quality_encoder(x):
    return {'Excellent':0,'Good':1,'Pass':2,'Fail':3}[x]
train['label'] = train.Quality_label.apply(quality_encoder)
train['label_Excellent'] = 1*(train['label'] == 0)
train['label_Good'] = 1*(train['label'] == 1)
train['label_Pass'] = 1*(train['label'] == 2)
train['label_Fail'] = 1*(train['label'] == 3)


train['prob_Excellent'] = 0.0
train['prob_Good'] = 0.0
train['prob_Pass'] = 0.0
train['prob_Fail'] = 0.0

In [14]:
# 预处理： 利用Attribute进行样本清洗
features = ['Attribute1', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5','Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Attribute10']
params = {'objective':'multiclass', 'num_class':4,
          'boosting':'gbdt','num_iterations':400,'learning_rate':0.1,'num_leaves':20}

kfold=KFold(n_splits=5, shuffle=True,random_state=829)
split=kfold.split(train,train.label)
model_stage1=[]
result_stage1=[]
for train_index,valid_index in split:
    
    X_train=train.iloc[train_index,:].copy()
    X_valid=train.iloc[valid_index,:].copy()
    
    model = CatBoostClassifier(iterations=600,depth=6,learning_rate=0.1,
                               verbose=100,
                               loss_function='MultiClass')
    model.fit(X_train.loc[:,features].values,X_train.label.values)
    X_valid.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] = model.predict_proba(X_valid.loc[:,features])
    result_stage1.append(X_valid)
result_stage1=pd.concat(result_stage1)

0:	learn: 1.2195293	total: 79.5ms	remaining: 47.6s
100:	learn: 0.0822796	total: 4.8s	remaining: 23.7s
200:	learn: 0.0333371	total: 9.39s	remaining: 18.6s
300:	learn: 0.0192109	total: 14.1s	remaining: 14s
400:	learn: 0.0136334	total: 19s	remaining: 9.41s
500:	learn: 0.0103280	total: 24.1s	remaining: 4.76s
599:	learn: 0.0083755	total: 28.6s	remaining: 0us
0:	learn: 1.2143454	total: 30.7ms	remaining: 18.4s
100:	learn: 0.0774816	total: 4.63s	remaining: 22.9s
200:	learn: 0.0297588	total: 9.08s	remaining: 18s
300:	learn: 0.0178502	total: 13.9s	remaining: 13.8s
400:	learn: 0.0121954	total: 18.5s	remaining: 9.17s
500:	learn: 0.0090464	total: 23.2s	remaining: 4.59s
599:	learn: 0.0072301	total: 27.8s	remaining: 0us
0:	learn: 1.2198488	total: 26.2ms	remaining: 15.7s
100:	learn: 0.0801717	total: 4.58s	remaining: 22.7s
300:	learn: 0.0174714	total: 13.9s	remaining: 13.8s
400:	learn: 0.0119525	total: 18.6s	remaining: 9.22s
500:	learn: 0.0090042	total: 23.3s	remaining: 4.6s
599:	learn: 0.0071729	total

In [15]:
result_stage1['label_pred']=np.argmax(result_stage1.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']].values,axis=1)
print("Attibute线下五折交叉验证准确率:",(result_stage1['label_pred']==result_stage1['label']).mean())

Attibute线下五折交叉验证准确率: 0.9913333333333333


In [16]:
# 取parameter5-10，取log
# features = ['Parameter5', 'Parameter7', 'Parameter8', 'Parameter9', 'Parameter10']
# train_data = train
# test_data = test
# train_data[features] = np.log(train_data[features] + 1e-5)
# test_data[features] = np.log(test_data[features] + 1e-5)
# # 取对数也容易报错

# ''''''
# # 特征融合
# train_data['7_8'] = train_data['Parameter7'] + train_data['Parameter8']
# test_data['7_8'] = test_data['Parameter7'] + test_data['Parameter8']
# train_data['7_9'] = train_data['Parameter7'] + train_data['Parameter9']
# test_data['7_9'] = test_data['Parameter7'] + test_data['Parameter9']
# train_data['7_10'] = train_data['Parameter7'] + train_data['Parameter10']
# test_data['7_10'] = test_data['Parameter7'] + test_data['Parameter10']
# train_data['8_9'] = train_data['Parameter8'] + train_data['Parameter9']
# test_data['8_9'] = test_data['Parameter8'] + test_data['Parameter9']
# train_data['9_10'] = train_data['Parameter9'] + train_data['Parameter10']
# test_data['9_10'] = test_data['Parameter9'] + test_data['Parameter10']
# train_data['7-10'] = (train_data['Parameter7'] - train_data['Parameter10'])
# test_data['7-10'] = (test_data['Parameter7'] - test_data['Parameter10'])
# train_data['9%8'] = train_data['Parameter9'] % train_data['Parameter8']
# test_data['9%8'] = test_data['Parameter9'] % test_data['Parameter8']
# train_data['7/8'] = train_data['Parameter7'] / train_data['Parameter8']
# test_data['7/8'] = test_data['Parameter7'] / test_data['Parameter8']
# train_data['7%8'] = train_data['Parameter7'] % train_data['Parameter8']
# test_data['7%8'] = test_data['Parameter7'] % test_data['Parameter8']
# train_data['8/9'] = train_data['Parameter8'] / train_data['Parameter9']
# test_data['8/9'] = test_data['Parameter8'] / test_data['Parameter9']

# scale_features = [
#     # '7_9', '8_9', '7_8', '9_10', '7_10', '9%8', '7/8', '7%8', '8/9',
# 					#'5_10',# '5*10',
# 					#'10*10', '10/8'
# 				  #'8%9', '9%8*9', '5_6',
# 				  #'5*5', '7*7', '9*9', '8*8',
# 					#'8-9', '7-8', '8*8'
# 				  ]
# # 这些加上之后容易报错

# # 组合所有特征
# features = features + scale_features#+ group_feat
# # drop_feat = []

# # drop_feat = drop_feat + ['Parameter6', #'Parameter7', #'Parameter8', 'Parameter9',
# # 	#'8*8', , '7*7',
# # 						 #'7_9', '8_9', '9_10',
# # 						 #'5_6',
# # 						 #'mean_Parameter8_Parameter9', 'mean_Parameter9_Parameter8', 'mean_Parameter7_Parameter9', 'mean_Parameter7_Parameter8',
# # #'9*9', 'mean_Parameter8_Parameter9', '7%8', '9%8','8/9', '9%8*9', '7_8',
# # 						 ]
# # for f in drop_feat:
# # 	if f in features:
# # 		features.remove(f)

# print('all train features:', features)

In [17]:
def lightgbm_model():
	model = LGBMClassifier(max_depth=5, learning_rate=0.01, n_estimators=1500, num_leaves=16,
						   objective='multiclass',
						   silent=True,
						   #reg_lambda=4., #reg_alpha=1.,
						   #bagging_fraction=0.9, feature_fraction=0.9,
						   )
	return model

def xgboost_model():
	model = XGBClassifier(max_depth=5, n_estimators=800, learning_rate=0.01, silent=True,
						  objective='multi:softmax',
						  )
	return model

def catboost_model():
	cbt_model = cbt.CatBoostClassifier(iterations=100,depth=10,learning_rate=0.1,
                               verbose=100,
                               loss_function='MultiClass'
	)
	return cbt_model

def adaboost_model():
	ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3), n_estimators=50, random_state=829)
	return ada_model

In [22]:
# 模型预测
features = ['Parameter5','Parameter7','Parameter8','Parameter9','Parameter10']
# train = train_data
# test = test_data
kfold=KFold(n_splits=5, shuffle=False)
split=kfold.split(train,train.label)
model_5fold=[]
result=[]
for train_index,valid_index in split:
    
    X_train=result_stage1.iloc[train_index,:].copy()
    X_train = X_train[X_train.label==X_train.label_pred].copy() # 过滤掉1%的样本
    X_valid=result_stage1.iloc[valid_index,:].copy()
    # model = CatBoostClassifier(iterations=100,depth=10,learning_rate=0.1,
    #                           verbose=100,
    #                           loss_function='MultiClass')
    model = StackingClassifier(
					classifiers=[xgboost_model(), catboost_model(), lightgbm_model(),
								#  adaboost_model()
								 ],
					use_probas=True, average_probas=False, meta_classifier=lr)
    model.fit(X_train.loc[:,features].values,X_train.label.values,)
    model_5fold.append(model)
    X_valid.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] = model.predict_proba(X_valid.loc[:,features].values)
    # 如果前面加了.value后面也要记得加上
    result.append(X_valid)
result=pd.concat(result)

0:	learn: 1.3350491	total: 26.4ms	remaining: 2.61s
99:	learn: 0.9830611	total: 2.58s	remaining: 0us


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0:	learn: 1.3351301	total: 24.2ms	remaining: 2.4s
99:	learn: 0.9833488	total: 2.05s	remaining: 0us


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0:	learn: 1.3393210	total: 24.1ms	remaining: 2.39s
99:	learn: 0.9975893	total: 2.27s	remaining: 0us


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0:	learn: 1.3355473	total: 27.2ms	remaining: 2.69s
99:	learn: 0.9877787	total: 2.48s	remaining: 0us


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0:	learn: 1.3367842	total: 16.9ms	remaining: 1.67s
99:	learn: 0.9888876	total: 3.05s	remaining: 0us


/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/envs/python35-paddle120-env/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [19]:
mean = []
for group in range(100):
    temp = result.groupby(['group_%s'%group],as_index=False)['prob_Excellent','prob_Good','prob_Pass','prob_Fail','label_Excellent','label_Good','label_Pass','label_Fail'].mean()
    a = np.abs(temp.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']].values
               - temp.loc[:,['label_Excellent','label_Good','label_Pass','label_Fail']].values).mean()
    mean.append(1/(1+10*a))
print("线下mae评估：",np.mean(mean),np.std(mean))
# 线下mae评估： 0.7071568484305453 0.01181459289243274
# 为什么线下总是这个

In [20]:
test['prob_Excellent'] = 0.0
test['prob_Good'] = 0.0
test['prob_Pass'] = 0.0
test['prob_Fail'] = 0.0

# 线上预测
for i in range(5):
    test.loc[:,['prob_Excellent','prob_Good','prob_Pass','prob_Fail']] += 0.2*model_5fold[i].predict_proba(test.loc[:,features].values)


prediction = test.groupby(['Group'],as_index=False)['prob_Excellent','prob_Good','prob_Pass','prob_Fail'].mean()
    
# 后处理
# for label in ['Excellent','Good','Pass','Fail']:
#     prediction['prob_%s'%label] = np.round(prediction['prob_%s'%label]/0.02)*0.02

prediction = prediction.loc[:,['Group','prob_Excellent','prob_Good','prob_Pass','prob_Fail']]
prediction.columns = ['Group','Excellent ratio','Good ratio','Pass ratio','Fail ratio']
prediction.to_csv('/home/aistudio/work/result.csv',index=False)

In [21]:
# grid_search('abc')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 